In [3]:
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from matplotlib import pyplot
import re
from sklearn import preprocessing
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import langid
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [4]:
posts = pd.read_csv('InstagramDatasetFinal.csv')

In [9]:
caption = pd.read_csv('CleanCaptionDF.csv',
                 lineterminator='\n')

In [11]:
posts.head(20)

,biography,business_category_name,connected_fb_page,country_block,edge_felix_video_timeline,edge_follow,edge_followed_by,edge_media_collections,edge_owner_to_timeline_media,edge_saved_media,...,Home Services,Lifestyle Services,Local Events,Non-Profits & Religious Organizations,Personal Goods & General Merchandise Stores,Professional Services,Publishers,Transportation & Accomodation Services,mean_likes,mean_comments
0,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
1,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
2,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
3,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
4,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
5,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
6,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
7,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
8,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727
9,NaN,NaN,NaN,False,0,11,1927598,0,19,0,...,0,0,0,0,0,0,0,0,759069.545455,25191.272727


In [12]:
caption1 = posts[['edge_media_to_caption']].copy()

In [13]:
caption1.head(20)

,edge_media_to_caption
0,O'CLOCK이 발매되었습니다!!! 여러분들의 많은 관심과 사랑부탁드릴께요❤️
1,즐거운 화이트데이~~메이트데이~~~🧛‍♂️❤️
2,O'CLOCK 곧 12시죠~! 두구두구 티저 이미지 공개! 😆
3,None
4,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요?
5,빼꼼! 메이 사랑해요~~~
6,복 많이받으세요!!!!❤️😆
7,이젠 어렵지 않아요
8,다음번에는 예고없이 바로 방송킬꺼에요🤨
9,곧 만나요🐰 먕먕~~!


In [14]:
hashtag_removed = re.sub(r'#[A-Za-z0-9]+','',caption1['edge_media_to_caption'][12])

In [15]:
hashtag_removed

'Hack the game. Unlock New Levels.  '

In [16]:
from nltk.corpus import stopwords
sw = stopwords.words('english')
np.array(sw)
def stopwords(row):
    words = []
    text = [word.lower()for word in str(row['english']).split() if word.lower() not in sw]
    words.extend(text)
    words = sorted(list(set(words)))
    return words
caption['token'] = caption.apply(stopwords, axis=1)

KeyError: 'english'

In [17]:
#block to clean caption text

sw = stopwords.words('english')
np.array(sw)
caption1['caption'] = ''
translator = str.maketrans('', '', string.punctuation)
stemmer= PorterStemmer()
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002700-\U000027BF"
        u"\U00002A00-\U00002AFF"
        u"\U00002600-\U000026FF"
                           "]+", flags=re.UNICODE)
for i in range( 0, caption1['edge_media_to_caption'].size ):
    if( (i+1)%1000 == 0 ):
        print(str(round(i/caption1['edge_media_to_caption'].size*100))+"% done")
    stem_result = []    
    input_str = str(caption1['edge_media_to_caption'][i]).lower() #convert to lowercase
    input_str = emoji_pattern.sub(r'', input_str) #remove emojis
    input_str = re.sub(r'#[A-Za-z0-9]+','',str(input_str)) #remove hashtags
    input_str = re.sub(r'@[A-Za-z0-9]+','',str(input_str)) #remove mentions
    
    #input_str = input_str.translate(translator) # remove punctuation #come back here
    #tokens = word_tokenize(input_str)
    #result = [i for i in tokens if not i in sw] # remove stop words
    #for word in result:
        #stem_result.append(stemmer.stem(word)) #get stem words
    #str1 = ' '.join(stem_result)
    caption1['caption'][i] = input_str
    

AttributeError: 'function' object has no attribute 'words'

In [ ]:
caption1.head(20)

In [ ]:
emoji_regexp = u'[\U00002600-\U000026FF]|[\U00002700-\U000027BF]|[\U0001f300-\U0001f5fF]|[\U0001f600-\U0001f64F]|' \
               u'[\U0001f680-\U0001f6FF]|[\U00002A00-\U00002AFF]|[\U0001F1E0-\U0001F1FF]'
re.findall(emoji_regexp, caption['edge_media_to_caption'][2], re.UNICODE)

In [ ]:
#caption1['caption'] = re.sub(r'#[A-Za-z0-9]+','',str(caption1['edge_media_to_caption']))
#caption1["caption"] = caption1['edge_media_to_caption'].str.replace('[^\w\s]','')
#caption1["caption"] = caption1['caption'].fillna(" ", inplace = True)
#caption1["caption"] = caption1['caption'].astype(str)

In [ ]:
caption1["caption"] = caption1['caption'].str.replace('[^\w\s]','') #remove emojis
caption1.head(20)

In [ ]:
def identifyLanguage(row):
    b = TextBlob(str(row['caption']))
    if(len(b) < 3):
        return 'en'
    else:
        return b.detect_language()
#depends on if TextBlob wants to work

In [ ]:
def identify_language(row):
    lang = langid.classify(row['caption'])
    return lang[0]
#depends on if langid wants to work

In [ ]:
caption1['language'] = caption1.apply(identify_language, axis=1)

In [ ]:
caption1.head()

In [ ]:
from googletrans import Translator
import pandas as pd
#from py_translator import Translator
translator = Translator()
#df = pd.DataFrame({'Spanish':['piso','cama']})
#df['English'] = df['Spanish'].apply(translator.translate, src='es', dest='en').apply(getattr, args=('text',))

In [ ]:
word = translator.translate(caption['caption'][0], src='ko', dest='en').text

In [ ]:
def translate(row):
    translator = Translator()
    if str(row['language']) == 'en':
        return str(row['caption'])
    else:
        #return ''
        try:
            english = translator.translate(str(row['caption']), src=row['language'], dest='en').text
            return english
        except:
            return ''

In [ ]:
caption1['english'] = ""
#translation to English
for i in range( 0, caption1['caption'].size):
    if( (i+1)%1000 == 0 ):
        print(str(round(i/caption1['caption'].size*100))+"% done")
    if(str(caption1['language'][i]) == 'en'):
        caption1['english'][i] = str(caption1['caption'][i])
    else:
        try:
            english = translator.translate(str(caption1['caption'][i]), src=caption1['language'][i], dest='en').text
            caption1['english'][i] = english
        except:
            caption1['english'][i] = ''

In [ ]:
caption['english'] = caption.apply(translate, axis=1)

In [ ]:
caption1 = caption1.fillna('')

In [ ]:
caption1.head(20)

In [ ]:
caption2 = caption1.copy()

In [ ]:
caption2 = caption2.drop(columns=['caption','english'])

In [ ]:
caption2.head(20)

In [ ]:
caption2.to_csv('CleanCaptionDF.csv')

In [ ]:
caption1.info()

In [ ]:
caption.to_csv('CaptionDF.csv')

# Start of NLP Processing

In [ ]:
caption = pd.read_csv('CaptionDF.csv')

In [ ]:
caption.info()

In [ ]:
import csv
with open(r"CleanCaptionDF.csv", 'rb') as f:
    reader = csv.reader(f)
    linenumber = 1
    try:
        for row in reader:
            linenumber += 1
    except Exception as e:
        print ("Error line %d: %s %s" % (linenumber, str(type(e)), e.message))

In [ ]:
caption = pd.read_csv('CleanCaptionDF.csv')

In [ ]:
caption.head()

In [ ]:
sw = stopwords.words('english')
np.array(sw)
caption1['caption'] = ''
translator = str.maketrans('', '', string.punctuation)
stemmer= PorterStemmer()

for i in range( 0, caption1['edge_media_to_caption'].size ):
    if( (i+1)%1000 == 0 ):
        print(str(round(i/caption1['edge_media_to_caption'].size*100))+"% done")
    stem_result = []    
    input_str = str(caption1['edge_media_to_caption'][i]).lower() #convert to lowercase
    #input_str = emoji_pattern.sub(r'', input_str) #remove emojis
    #input_str = re.sub(r'#[A-Za-z0-9]+','',str(input_str)) #remove hashtags
    #input_str = re.sub(r'@[A-Za-z0-9]+','',str(input_str)) #remove mentions
    
    input_str = input_str.translate(translator) # remove punctuation #come back here
    tokens = word_tokenize(input_str)
    result = [i for i in tokens if not i in sw] # remove stop words
    for word in result:
        stem_result.append(stemmer.stem(word)) #get stem words
    str1 = ' '.join(stem_result)
    caption1['caption'][i] = input_str

In [ ]:
from nltk.corpus import stopwords
sw = stopwords.words('english')
np.array(sw)
def stopwords(row):
    words = []
    text = [word.lower()for word in str(row['english']).split() if word.lower() not in sw]
    words.extend(text)
    words = sorted(list(set(words)))
    return words
caption['token'] = caption.apply(stopwords, axis=1)

In [ ]:
caption['english'] = caption['english'].replace(np.nan, '', regex=True)

In [ ]:
caption.head(20)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [ ]:
count_vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 100) 

In [ ]:
sentences = []
for i in range( 0, caption['english'].size ):
    if( (i+1)%1000 == 0 ):
        print(str(round(i/30000*100))+"% done")
    sentences.append(caption['english'][i])

In [ ]:
word_features = count_vectorizer.fit_transform(sentences)

#tfidf_transformer = TfidfTransformer()
#clean_features = tfidf_transformer.fit_transform(clean_features)

# Numpy arrays are easy to work with, so convert the result to an 
# array
word_features = word_features.toarray()

In [ ]:
vocab = count_vectorizer.get_feature_names()
word_features = pd.DataFrame(data=word_features, columns=vocab)
len(word_features)

In [ ]:
from textblob import TextBlob

In [ ]:
caption2['sentiment'] = ''
for i in range( 0, caption2['english'].size):
    if( (i+1)%1000 == 0 ):
        print(str(round(i/caption2['english'].size*100))+"% done")
    analysis = TextBlob(caption2['english'][i])
    caption2['sentiment'][i] = analysis.sentiment[0]

In [ ]:
caption2.head()

In [ ]:
caption['language'] = caption['language'].replace(np.nan, 'en', regex=True)
caption['edge_media_to_caption'] = caption['edge_media_to_caption'].replace(np.nan, 'en', regex=True)
caption['caption'] = caption['caption'].replace(np.nan, 'en', regex=True)

In [ ]:
caption = pd.get_dummies(caption,columns=['language'])

In [ ]:
caption2 = pd.get_dummies(caption2,columns=['language'])

In [ ]:
nlp_features = caption2.copy()
#nlp_features = caption.drop(caption.columns[cols],axis=1,inplace=True)

In [ ]:
cols = [0,1,2]
nlp_features.drop(nlp_features.columns[cols],axis=1,inplace=True)

In [ ]:
nlp_features.head()

In [ ]:
nlp_features.to_csv('NLPFeatures.csv')